# Chapter 3 : Write your first Agent

## The Source class

### The Source is the place where you implement the acquisition of your data

Regardless of the type of data, the framework will always consume a Source the same way.<br>
Many generic sources are provided by the framework and it's easy to create a new one.

### The Source strongly encourages to stream incoming data

Whatever the size of the dataset, memory will never be an issue.

### Example

Let's continue with our room sensors.

In [ ]:
# this source provides us with a CSV line each second
from pyngsi.sources.more_sources import SourceSampleOrion

# init the source
src = SourceSampleOrion()

# iterate over the source
for row in src:
    print(row)

Here we can see that a row is an instance of a Row class.<p>

For the vast majority of the Sources, the provider keeps the same value during the datasource lifetime.<br>
We'll go into details in next chapters.<p>

**In practice you won't iterate the Source by hand. The framework will do it for you.**

## The Agent class

Here comes the power of the framework.<br>
By using an Agent you will delegate the processing of the Source to the framework.<p>

Basically an Agent needs a Source for input and a Sink for output.<br>
It also needs a function in order to convert incoming rows to NGSI entities.<p>

Once the Agent is initialized, you can run it !

Let's continue with our rooms.

In [ ]:
from pyngsi.sources.more_sources import SourceSampleOrion
from pyngsi.sink import SinkStdout
from pyngsi.agent import NgsiAgent

# init the source
src = SourceSampleOrion()

# for the convenience of the demo, the sink is the standard output
sink = SinkStdout()

# init the agent
agent = NgsiAgent.create_agent(src, sink)

#run the agent
agent.run()

Here you can see that incoming rows are outputted 'as is'.<br>
It's possible because SinkStdout ouputs whatever it receives on its input.<br>
But SinkOrion expects valid NGSI entities on its input.<p>

So let's define a conversion function.

In [ ]:
from pyngsi.sources.source import Row
from pyngsi.ngsi import DataModel

def build_entity(row: Row) -> DataModel:
    id, temperature, pressure = row.record.split(';')
    m = DataModel(id=id, type="Room")
    m.add("dataProvider", row.provider)
    m.add("temperature", float(temperature))
    m.add("pressure", int(pressure))
    return m

And use it in the Agent.

In [ ]:
# init the Agent with the conversion function
agent = NgsiAgent.create_agent(src, sink, process=build_entity)

# run the agent
agent.run()

# obtain the statistics
print(agent.stats)

Congratulations ! You have developed your first pyngsi Agent !<p>

Feel free to try SinkOrion instead of SinkStdout.<br>
Note that you get statistics for free.

Inside your conversion function you can filter input rows just by returning None.<br>
For example, if you're not interested in Room3 you could write this function.

In [ ]:
def build_entity(row: Row) -> DataModel:
    id, temperature, pressure = row.record.split(';')
    if id == "Room3":
        return None
    m = DataModel(id=id, type="Room")
    m.add("dataProvider", row.provider)
    m.add("temperature", float(temperature))
    m.add("pressure", int(pressure))
    return m

agent = NgsiAgent.create_agent(src, sink, process=build_entity)
agent.run()
print(agent.stats)

### The side_effect() function

As of v1.2.5 the Agent takes the `side_effect()` function as an optional argument.<br>
That function will allow to create entities aside of the main flow. Few use cases might need it.

In [ ]:
def side_effect(row, sink, model) -> int:
    # we can use as an input the current row or the model returned by our process() function
    m = DataModel(id=f"Building:MainBuilding:Room:{model['id']}", type="Room")
    sink.write(m.json())
    return 1 # number of entities created in the function

agent = NgsiAgent.create_agent(src, sink, process=build_entity, side_effect=side_effect)
agent.run()
print(agent.stats)

## Conclusion

This NGSI Agent developed in this chapter is a very basic one.<p>
Principles remain the same when things become more complex.